# Process ADB MRIO Excel files

This notebook converts the ADB MRIO Excel files into machine-readable format and saves them as space-efficient parquet files. These are stored in `data/interim/`. 

In [1]:
import pandas as pd
import os
import re

In [2]:
def process_table(df):
    
    # Remove the last row and the first 2 columns
    df = df.drop(df.index[-1])
    df = df.iloc[:, 2:]

    # Collapse MultiIndex headers into one
    df.columns = [f'{level_1}_{level_2}' for level_1, level_2 in df.columns]

    # Rename the ToT column
    colnames = df.columns.tolist()
    mapping = {colnames[-1]: 'ToT'}
    df = df.rename(columns=mapping)

    # Fix row labels
    rowlabels = [f"{c}_{d}" if not (pd.isna(c) or c == 'ToT') else d for c, d in zip(df.iloc[:, 0], df.iloc[:, 1])]
    df.insert(2, '', rowlabels)
    df = df.iloc[:, 2:]
    
    # Drop intermediates totals
    df = df.drop(df[df[''] == 'r60'].index)

    # Replace blank cells with zero
    df = df.replace(' ', 0)

    return df

In [3]:
def load_and_save(inputfolder, outputfolder, outputfile, outputsuffix=None):
    
    filelist = [file for file in os.listdir(f'../data/raw/{inputfolder}') if not file.startswith('.')]

    for file in filelist:
        mrio = pd.read_excel(
            f'../data/raw/{folder}/{file}',
            skiprows=5,
            header=[0,1]
        )
        mrio = process_table(mrio)
        year = re.search('[0-9]{4}', file).group()
        
        if outputsuffix is None:
            mrio.to_parquet(f'../data/mrio/{outputfolder}/{outputfile}-{year}.parquet', index=False)
        else:
            mrio.to_parquet(f'../data/mrio/{outputfolder}/{outputfile}-{year}{outputsuffix}.parquet', index=False)
        
        print(f'{year} done')

## ADB MRIO 72 economies

In [4]:
folder = 'ADB MRIO, 72 economies'
load_and_save(inputfolder=folder, outputfolder='ADB-MRIO', outputfile='ADB-MRIO')

2021 done
2019 done
2022 done
2018 done
2017 done
2020 done


## ADB MRIO 62 economies

In [54]:
folder = 'ADB MRIO, 62 economies'
load_and_save(inputfolder=folder, outputfolder='ADB-MRIO62', outputfile='ADB-MRIO62')

2008 done
2009 done
2011 done
2018 done
2016 done
2019 done
2010 done
2012 done
2015 done
2014 done
2013 done
2017 done
2007 done
2021 done
2020 done
2000 done


## ADB MRIO 72 economies (version Jun 2023)

In [55]:
folder = '9 MRIO 2020-2022 for upload (ao Jun 2023)'
load_and_save(inputfolder=folder, outputfolder='ADB-MRIO_jun2023', outputfile='ADB-MRIO', outputsuffix='_jun2023')

2021 done
2022 done
2020 done
